In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from importlib import reload
from PATH import data_path
import utils
import Model
import re

In [2]:
from sklearn.preprocessing import scale

In [3]:
%matplotlib widget

## get data

In [4]:
DF,y = utils.get_known_sample(data_path,'ER')        # labeled data
ER_to_pre = utils.get_unknown_sample(data_path,'ER') # unlabeled
# make it array
data = DF.values 

In [5]:
# three more preprocess
data_norm = scale(data)
data_log = scale(np.log(data+0.01))
data_erbb2 = utils.scaling_by_gene(data,'ERBB2',column=DF.columns)
# combine into a list
datas = [data,data_norm,data_log,data_erbb2]

scale by gene ERBB2 in column 13739


##  used trained model to predict unknown data

In [6]:
import logger
reload(logger)

<module 'logger' from '/home/ZwZ/script/HER2_prediction/logger.py'>

In [7]:
logg = logger.Mlog('ER_result/RF.log')

In [8]:
all_perform=logg.all_perform

In [9]:
val_sum = all_perform[:,:,0] + all_perform[:,:,1]

### pick the most performance

In [10]:
np.argmax(val_sum,axis=0)

array([131, 131, 348,  19])

In [11]:
np.max(val_sum,axis=0)

array([2.     , 2.     , 2.     , 1.98459])

In [12]:
logg.performance_of(214)

,V_F1,V_AUCROC,T_F1,T_AUCROC
PREPROCESS,,,,
raw data,0.96364,0.97685,0.94979,0.95877
log data,0.96364,0.98056,0.95378,0.94309
norm data,0.96364,0.98380,0.94737,0.94564
erbb2 data,0.95238,0.97824,0.92562,0.90340


In [13]:
logg.performance_of(290)

,V_F1,V_AUCROC,T_F1,T_AUCROC
PREPROCESS,,,,
raw data,0.95726,0.96667,0.95547,0.95130
log data,0.96610,0.96726,0.95635,0.94655
norm data,0.95000,0.97560,0.96177,0.95013
erbb2 data,0.91667,0.89583,0.92495,0.93549


In [14]:
logg.performance_of(194)

,V_F1,V_AUCROC,T_F1,T_AUCROC
PREPROCESS,,,,
raw data,0.93333,0.89514,0.94501,0.94312
log data,0.92453,0.89045,0.93522,0.94472
norm data,0.91589,0.89514,0.94400,0.93585
erbb2 data,0.87037,0.86104,0.91849,0.91216


## PICK 290 !!

In [15]:
seed = logg.seed_of(290)

## PCA decomposition

In [16]:
from sklearn import discriminant_analysis as DA

In [17]:
from sklearn.decomposition import PCA,KernelPCA

In [18]:
DF,y = utils.get_known_sample(data_path,'HER2')        # labeled data
ER_to_pre = utils.get_unknown_sample(data_path,'HER2') # unlabeled
# make it array
data = DF.values 

In [19]:
# three more preprocess
data_norm = scale(data)
data_log = scale(np.log(data+0.01))
data_erbb2 = utils.scaling_by_gene(data,'ERBB2',column=DF.columns)
# combine into a list
datas = [data,data_norm,data_log,data_erbb2]

scale by gene ERBB2 in column 13739


/home/ZwZ/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [20]:


#data_2d = KernelPCA(n_components=2,kernel='rbf').fit_transform(data_log)

#data_2d=lad_ls[0]._model.transform(data)

data_2d=PCA(n_components=2).fit_transform(data_log)

plt.figure(figsize=(12,10))
plt.scatter(data_2d[:,0],data_2d[:,1],c=y,s=8,marker='X');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## t-SNE

In [21]:
from sklearn.manifold import TSNE

In [22]:
from sklearn.decomposition import TruncatedSVD

In [23]:
#data_pca = PCA(n_components=50).fit_transform(data_erbb2)
data_pca = KernelPCA(n_components=2,kernel='rbf').fit_transform(data_erbb2)
#data_pca = TruncatedSVD(n_components=50).fit_transform(data)
data_sne = np.stack([TSNE(perplexity=i,).fit_transform(data_pca) for i in range(0,55,5)])

In [118]:
np.save('kpca_tsne_erbb2_p5-50.npy',data_sne)

In [24]:
data_sne.shape

(11, 611, 2)

In [29]:
plt.figure()

for i in range(data_sne.shape[0]):
    plt.subplot(4,3,1+i)
    plt.scatter(data_sne[i][:,0],data_sne[i][:,1],c=y,s=3);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …